In [79]:
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from pathlib import Path

In [2]:
%matplotlib qt

In [3]:
class Mols(Enum):
    dmag = "dmag"
    ocm = "ocm"
    butanol = "butanol"
    peta = "peta"


class W(Enum):
    x = "x"
    mass = "w_mass"
    vol = "w_vol"

In [6]:
D_mean = 15000
DF=pd.read_csv('Datas/Viscosity_all_data.csv',index_col=0)
DF.head(2)

,compound,x,Temperature,rho,w_mass,w_vol,time,Viscosity,Kin_viscosity,D,D0,E
0,butanol,100.0,13.0,0.82668,100.0,100.0,127.760000,3.700000,4.475734,77337.837838,2.942474e+08,19660.651841
1,butanol,100.0,13.1,0.82668,100.0,100.0,87.724138,3.691379,4.465306,77545.607003,2.942474e+08,19660.651841


In [91]:
# D-T plots
def plot_DT(compound: Mols, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (df["compound"] == Mols.butanol.value)

    data = df[compound_filter]
    grouped = data.groupby(by=[w.value])
    colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

    fig, ax = plt.subplots(num="DT")
    for _, group in grouped:
        x = group[w.value].mean()
        T = group["Temperature"]
        color = colors_list.pop()

        D = group["D"] / D_mean
        E = group["E"].mean()
        D0 = group["D0"].mean() / D_mean
        D_ols = D0 * np.exp(-E / (8.314 * (T + 273.15)))
        ax.plot(T, D_ols, label=f"{x: 5.2f} %", c=color)

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - T ({w.value})")
        ax.set_xlabel("Temperature [C]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


def plot_VT(compound: Mols, w: W,save_dir=None):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (df["compound"] == Mols.butanol.value)

    data = df[compound_filter]
    grouped = data.groupby(by=[w.value])
    colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

    fig, ax = plt.subplots(num="VT")
    for _, group in grouped:
        x = group[w.value].mean()
        T = group["Temperature"]
        color = colors_list.pop()

        V = group["Viscosity"]
        # E = group["E"].mean()
        # D0 = group["D0"].mean() / D_mean
        # D_ols = D0 * np.exp(-E / (8.314 * (T + 273.15)))
        ax.plot(T, V, label=f"{x: 5.2f} %", c=color)

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Viscosity - T ({w.value})")
        ax.set_xlabel("Temperature [C]")
        ax.set_ylabel("Viscosity [cP]")

    title = f'V-T({compound.value.upper()}-BUT)'
    plt.legend()
    if save_dir:
        fig.savefig(save_dir/f'{title}.jpg', dpi=600)
    plt.show()


# D-x plots
def plot_Dx(compound: Mols, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (df["compound"] == Mols.butanol.value)
    data = df[compound_filter]
    temper_list = np.arange(15, 41, 5)
    colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

    fig, ax = plt.subplots(num="DX")
    for T in temper_list:
        cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
        group = data[cond]
        group = group[[w.value, "D"]].groupby(by=w.value).mean().reset_index()

        x = group[w.value]
        color = colors_list.pop()

        D = group["D"] / D_mean
        ax.plot(x, D, c=color, label=f"{T: 3.0f} C")

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - {w.value} (T)")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


# D-x plots
def plot_Vx(compound: Mols, w: W,save_dir=None):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (df["compound"] == Mols.butanol.value)
    data = df[compound_filter]
    temper_list = np.arange(15, 41, 5)
    colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

    fig, ax = plt.subplots(num="Vx")
    for T in temper_list:
        cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
        group = data[cond]
        group = group[[w.value, "Viscosity"]].groupby(by=w.value).mean().reset_index()

        x = group[w.value]
        color = colors_list.pop()
        V = group["Viscosity"]
        ax.plot(x, V, c=color, label=f"{T: 3.0f} C")

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Viscosity - {w.value} (T)")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("Viscosity [cP]")

        title = f'V-{w.name}({compound.value.upper()}-BUT)'
    plt.legend()
    if save_dir:
        fig.savefig(save_dir/f'{title}.jpg', dpi=600)

    plt.show()


# E-x plots
def plot_Ex(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots(num="E")
    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["E", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["E"] / 1000
        ax.plot(
            [x.iloc[0], 100],
            [E.iloc[0], 19.660651841],
            # label=f"{name}",
        )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"E - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("E [kJ]")
    plt.legend()
    plt.show()


def plot_rho(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots()

    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["rho", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["rho"]
        # ax.plot(
        #     [w.iloc[0],w.iloc[-1]],
        #     [E.iloc[0],E.iloc[-1]],
        #     # label=f"{name}",
        # )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"Rho - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("rho [g/ml]")
    plt.legend()
    plt.show()

In [94]:
plot_VT(save_dir=Path(r'Plots\v_plots'),compound=Mols.dmag,w=W.x)

In [47]:

from Experiments.base import Experiment
from Experiments import functions
from statsmodels.api import OLS

In [56]:
group = list(DF.groupby(by=["compound"]))[0][1]

In [57]:
a=Experiment(folder='')

In [58]:
a.d=group

In [59]:
a.d.rename(
    columns={'x':'x2','Temperature':'x','Viscosity':'y'},inplace=True
)

In [54]:
def regress(experiment: Experiment):
    experiment = experiment.apply(functions.C_to_K).apply(functions.linearize)

    df = experiment.d
    df["x0"] = 1
    result = OLS(df["y"], df[["x", "x0"]]).fit()
    means = result.params

    D0 = np.exp(means["x0"])
    E = -8.314 * means["x"]

    conf_int = result.conf_int(0.005).loc
    conf_int["x0"] = np.exp(conf_int["x0"])
    dD0 = (conf_int["x0"].max() - conf_int["x0"].min()) / 2
    conf_int["x"] = -8.314 * conf_int["x"]
    dE = (conf_int["x"].max() - conf_int["x"].min()) / 2

    info = dict(
        E=E,
        D0=D0,
        dD0=dD0,
        dE=dE,
        f_statistic=result.fvalue,
        r2=result.rsquared,
    )
    func = lambda T: D0 * np.exp(-E / (8.314 * T))
    return info, result, func


def create_OLS(exp: Experiment):
    info, result, func = regress(exp)
    exp.set_info(**info)
    x = np.linspace(13, 42, 100) + 273.15
    ols_res = Experiment(
        d=pd.DataFrame(
            {
                "x": x,
                "y": func(x),
                "time": x * 0,
            }
        ),
        name ="interpolated",
        folder ='s'
    )
    return ols_res.apply(functions.K_to_C)


In [60]:
b= create_OLS(a)

In [62]:
b.d

,x,y,time
0,13.000000,3.788909,0.0
1,13.292929,3.760901,0.0
2,13.585859,3.733156,0.0
3,13.878788,3.705672,0.0
4,14.171717,3.678446,0.0
...,...,...,...
95,40.828283,1.991790,0.0
96,41.121212,1.979552,0.0
97,41.414141,1.967411,0.0
98,41.707071,1.955368,0.0


In [67]:
VERBOSE_COLORS = {
    'OK': 'g',
    'OK_inner': 'b',
    'image_sweep_check': 'r',
    'combine_check': 'w',
}
def comparation_plot(
    experiment: Experiment,
    ols_exp: Experiment,
    title,
    xlabel,
    ylabel,
    interactive=False,
    save_folder=None,
):
    fig, ax = plt.subplots()
    ax.scatter(
        experiment.d['x'],
        experiment.d['y'],
        marker='.',
        label='Data',
        alpha=0.6,
    )

    E = experiment.info['E']
    D0 = experiment.info['D0']
    ax.plot(
        ols_exp.d['x'],
        ols_exp.d['y'],
        color='black',
        label=f'OLS: \nE= {E/1000: >8.2f} kJ\nD= {D0: >8.2e} m2/s',
    )

    fig.canvas.manager.set_window_title(title + ' plot')
    fig.subplots_adjust(
        top=0.9,
        bottom=0.1,
        left=0.1,
        right=0.9,
        hspace=0.2,
        wspace=0.2,
    )

    # ax.set_title(f"{experiment.name}: ({experiment.info['w']}% mass)")
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()

    if interactive:
        plt.show()
    if save_folder is not None:
        os.makedirs(f'{save_folder}\Plots', exist_ok=True)
        fig.savefig(f'{save_folder}\Plots\\{title}_{experiment.name}.jpg', dpi=600)

In [68]:
comparation_plot(
    a,
    b,
    title="OLS_Diffusion",
    xlabel="Temperature [C]",
    ylabel="V [cp]",
    interactive=True,
)

In [42]:
# a.apply(functions.C_to_K)
# a.apply(functions.linearize)

,compound,x,Temperature,rho,w_mass,w_vol,time,Viscosity,Kin_viscosity,D,D0,E
0,butanol,100.0,13.0,0.82668,100.0,100.0,127.760000,3.700000,4.475734,77337.837838,2.942474e+08,19660.651841
1,butanol,100.0,13.1,0.82668,100.0,100.0,87.724138,3.691379,4.465306,77545.607003,2.942474e+08,19660.651841
2,butanol,100.0,13.2,0.82668,100.0,100.0,101.250000,3.677500,4.448517,77868.040976,2.942474e+08,19660.651841
3,butanol,100.0,13.3,0.82668,100.0,100.0,10.750000,3.640000,4.403155,78695.054945,2.942474e+08,19660.651841
4,butanol,100.0,13.4,0.82668,100.0,100.0,199.500000,3.700000,4.475734,77445.945946,2.942474e+08,19660.651841
...,...,...,...,...,...,...,...,...,...,...,...,...
257,butanol,100.0,40.8,0.82668,100.0,100.0,5233.750000,2.010000,2.431412,156194.029851,2.942474e+08,19660.651841
258,butanol,100.0,40.9,0.82668,100.0,100.0,5293.000000,1.990000,2.407219,157814.070352,2.942474e+08,19660.651841
259,butanol,100.0,41.0,0.82668,100.0,100.0,5284.000000,1.990000,2.407219,157864.321608,2.942474e+08,19660.651841
260,butanol,100.0,41.1,0.82668,100.0,100.0,5276.500000,1.990000,2.407219,157914.572864,2.942474e+08,19660.651841


In [28]:
plot_Vx(compound=Mols.peta,w=W.x)